Импортируем библиотеки и считываем данные, разделив их на объекты и ответы

In [39]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

data = pd.read_csv('output/processed_data.csv', sep=';', index_col=0)

In [40]:
data.head()

,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,
id2875421,1,930.399753,0,455.0,1.500479
id2377394,0,930.399753,0,663.0,1.807119
id3858529,1,930.399753,0,2124.0,6.392080
id3504673,1,930.399753,0,429.0,1.487155
id2181028,1,930.399753,0,435.0,1.189925


Оптимизировать будем MSLE метрику

$$\text{MSLE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2$$

In [43]:
### Можно показать, что для оптимизации MSLE,
### достаточно логарифмировать таргетную переменную, 
### а потом оптимизировать привычную MSE
data = data.assign(log_trip_duration=np.log1p(data['trip_duration']))
data = data.drop(['trip_duration', 'store_and_fwd_flag'], axis=1)

Объяснение:

Пусть имеем модель $a(x)$, обученную на MSE от $y$:
$$
\text{MSE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(y_i - a(x_i)\big)^2 \rightarrow min
$$

Также представим модель $a^*(x)$, обученную на MSE от $\log{(y + 1)}$:
$$
\text{MSE}(X, \log{(y+1)}, a^*) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - a^*(x_i)\big)^2 \rightarrow min
$$

Так как вторая модель старается аппроксимироваться ответы в виде $a^*(x) \approx \log{(y+1)}$, то для того, чтобы получить изначальные таргеты, необходимо выходы $a^*(x)$ проэкспоненцировать в виде следующего отношения: $a(x) = e^{a^*(x)} - 1$, то есть $a^*(x) = \log{(a(x) + 1)}$, где $a^*(x)$ дает логарифмические выходы, а $a(x)$ - изначальные. Подставим данное соотношение в MSE выше, получим:

$$
\text{MSE}(X, \log{(y+1)}, \log{(a(x) + 1)}) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2 \rightarrow min
$$

А это в точности MSLE от $y$:

$$
\text{MSLE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2 \rightarrow min
$$

In [44]:
# Разделим данные на Train и Test
from sklearn.model_selection import train_test_split

X = data.drop('log_trip_duration', axis=1)
y = data['log_trip_duration']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [45]:
# Установим метод кросс-валидации
from sklearn.model_selection import KFold, cross_validate

splitter = KFold(n_splits=20, shuffle=True, random_state=33)

In [46]:
# Проведем кросс-валидацию
model = LinearRegression()

cv_result = cross_validate(model, X_train, y_train,
                           scoring='neg_mean_squared_error',
                           cv=splitter, return_train_score=True)

In [47]:
# Получим средннюю ошибку
np.mean(-cv_result['test_score'])

0.43132417223638636

Теперь построим модель на всей тренировочной выборке и замерим качество на тесте

In [48]:
model = LinearRegression()

model.fit(X_train, y_train)

np.mean((y_test - model.predict(X_test)) ** 2)

0.40746183478090053